**Zadanie 2. (8p)** W zadaniu tym powinieneś napisać wyszukiwarkę dla Wikipedii. Wymagamy
tu oddzielenia procesu indeksowania treści od wyszukiwania, czyli podczas indeksowania powinny
powstać listy postingowe, które następnie należy zapisać do bazy danych (lub do pliku). Proces
wyszukiwania z kolei powinien startować szybko i wczytywać listy postingowe do pamięci w sposób
leniwy (czyli tylko wtedy, gdy któraś jest potrzebna).
Wyszukiwarka powinna prezentować wyniki w kolejności uwzględniającej następujące rzeczy:

a) Trafienia w tytule są cenniejsze od trafienia poza tytułem

b) Trafienia dokładne (kotami-kotami) są cenniejsze od trafienia niedokładnego (czyli zgodności
lematu, np. (kotami-kotu).

c) Lekko preferowane są dokumenty o mniejszych identyfikatorach (czyli występujące wcześniej w
pliku z Wikipedyjką)

Do prezentacji wyników wykorzystaj kolory, w celu zwiększenia czytelności. Wyświetlaj zarówno
tytuł artykułu, jak i fragment jego treści, zawierający termy z zapytania. Wystarczy, że Twój
program zadziała dla zmniejszonej Wikipedii (patrz SKOS).

In [2]:
import csv
import logging
from collections import defaultdict
from itertools import chain

from redis import Redis

In [3]:
logging.basicConfig(level=logging.INFO)
DEBUG = True

In [4]:
BASE_FORMS_FILE_PATH = 'data/polimorfologik-2.1.txt'
WIKI_ARTICLES_FILE_PATH = 'data/fp_wiki.txt'

REDIS_DB_URL = 'redis://localhost/0'

In [7]:
BASE_FORMS = {}

with open(BASE_FORMS_FILE_PATH) as f:
    for base_form, word, *_ in csv.reader(f, delimiter=';'):
        BASE_FORMS[word.lower()] = base_form

In [8]:
WIKI_ARTICLES = []

with open(WIKI_ARTICLES_FILE_PATH) as f:
    lines = iter(f)
    try:
        while True:
            _title_with_prefix = next(lines)
            title = next(lines).split()
            text = []
            while sentence := next(lines).split():
                text.extend(sentence)
            WIKI_ARTICLES.append((title, text))
    except StopIteration:
        pass

# Indexer

In [9]:
def index_wiki_articles():
    index = defaultdict(set)
    for id_, (title, text) in enumerate(WIKI_ARTICLES):
        for word in chain(title, text):
            if base_form := BASE_FORMS.get(word.lower()):
                index[base_form].add(id_)
    return index

In [10]:
logging.info('Indexing Wiki articles')
index = index_wiki_articles()
logging.info('Saving index to Redis')
with Redis.from_url(REDIS_DB_URL) as r:
    for i, (base_form, article_ids) in enumerate(index.items()):
        posting_list = ','.join(str(id_) for id_ in article_ids)
        r.set(base_form, posting_list)
        if i > 0 and i % 10_000 == 0:
            logging.info(f'Saved {i} posting lists')

INFO:root:Indexing Wiki articles
INFO:root:Saving index to Redis
INFO:root:Saved 10000 posting lists
INFO:root:Saved 20000 posting lists
INFO:root:Saved 30000 posting lists
INFO:root:Saved 40000 posting lists
INFO:root:Saved 50000 posting lists
INFO:root:Saved 60000 posting lists
INFO:root:Saved 70000 posting lists
INFO:root:Saved 80000 posting lists
INFO:root:Saved 90000 posting lists
INFO:root:Saved 100000 posting lists
INFO:root:Saved 110000 posting lists


# Search engine

In [11]:
def base(words):
    try:
        return {BASE_FORMS[word.lower()] for word in words}
    except KeyError as e:
        raise ValueError(f'No base form for "{e.args[0]}"')

In [12]:
def find_articles(query):
    base_query = base(query)
    with Redis.from_url(REDIS_DB_URL) as r:
        posting_lists = r.mget(base_query)
    article_ids = set.intersection(*(
        {int(id_) for id_ in plist.decode().split(',')}
        for plist in posting_lists
    ))
    
    results = []
    for id_ in article_ids:
        title, text = WIKI_ARTICLES[id_]
        title_indices = [
            i for i, word in enumerate(title)
            if BASE_FORMS.get(word.lower()) in base_query
        ]
        text_indices = [
            i for i, word in enumerate(text)
            if BASE_FORMS.get(word.lower()) in base_query
        ]
        results.append((id_, (title, title_indices), (text, text_indices)))
    return results

In [13]:
TITLE_HITS_MODIFIER = 10
EXACT_MATCH_MODIFIER = 5
ARTICLE_ID_MODIFIER = -0.00001

def score(result, query):
    id_, (title, title_indices), (text, _) = result
    title_hits = len(title_indices)
    exact_matches = len([
        qword for qword in query
        if qword.lower() in {word.lower() for word in chain(title, text)}
    ])
    return (
        title_hits * TITLE_HITS_MODIFIER
        + exact_matches * EXACT_MATCH_MODIFIER
        + id_ * ARTICLE_ID_MODIFIER
    )

In [14]:
def rank_results(results, query):
    scored = [(result, score(result, query)) for result in results]
    return sorted(scored, key=lambda rs: rs[1], reverse=True)

In [15]:
def highlight(text):
    return f'\033[1m\033[34m{text}\033[m'

In [16]:
def display_result(result, score):
    _, (title, title_indices), (text, text_indices) = result
    for i in title_indices:
        title[i] = highlight(title[i])
    for i in text_indices:
        text[i] = highlight(text[i])
    return f"{highlight(score)} {' '.join(title)}\n{' '.join(text)}\n"

In [17]:
def search(query_raw):
    query = query_raw.split()
    results = find_articles(query)
    for result, score in rank_results(results, query):
        print(display_result(result, score))

In [20]:
search('programowanie logiczne')

29.85674 Programowanie logiczne
Programowanie logiczne ( nazywane także programowaniem w logice lub programowaniem w języku logiki ) - metoda programowania , będąca odmianą programowania deklaratywnego , w której program podawany jest jako pewien zestaw zależności , a obliczenia są dowodem pewnego twierdzenia w oparciu o te zależności . Na przykład chcemy stwierdzić , czy w danym grafie skierowanym istnieje ścieżka z pewnego punktu do pewnego innego punktu . Krawędzie zapisane są relacją edge ( Skąd , Dokąd ) . Nasz program wyglądałby w Prologu tak:

14.85673 Programowanie imperatywne
Programowanie imperatywne – paradygmat programowania , który opisuje proces wykonywania jako sekwencję instrukcji zmieniających stan programu . Podobnie jak tryb rozkazujący w lingwistyce wyraża żądania jakichś czynności do wykonania . Programy imperatywne składają się z ciągu komend do wykonania przez komputer . Rozszerzeniem ( w sensie wbudowanych funkcji ) i rodzajem ( w sensie paradygmatu ) programowa